In [ ]:
pip install tensorflow opencv-python numpy matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = '/content/drive/MyDrive/ISL_dataset'  # <-- Change this if needed

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical',
    shuffle=True,
    seed=42
)

class_names = list(train_data.class_indices.keys())
print("Classes:", class_names)


Found 970 images belonging to 8 classes.
Found 240 images belonging to 8 classes.
Classes: ['goodbye', 'hello', 'i_love_you', 'namaste', 'no', 'sorry', 'thank_you', 'yes']


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze for fast training

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,984 (9.24 MB)

 Trainable params: 165,000 (644.53 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

model.save("isl_model.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.4974 - loss: 1.5401 - val_accuracy: 0.9917 - val_loss: 0.0776
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9772 - loss: 0.1041 - val_accuracy: 0.9958 - val_loss: 0.0382
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.9834 - loss: 0.0473 - val_accuracy: 1.0000 - val_loss: 0.0128
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9912 - loss: 0.0371 - val_accuracy: 0.9958 - val_loss: 0.0148
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9933 - loss: 0.0233 - val_accuracy: 0.9958 - val_loss: 0.0099
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.9943 - loss: 0.0285 - val_accuracy: 0.9958 - val_loss: 0.0148
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.9989 - loss: 0.0149 - val_accuracy: 0.9958 - val_loss: 0.0115
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 1.0000 - loss: 0.0103 - val_accuracy: 0.9958 - val_loss: